**Aisha Muhammad Nawaz  L200921**  
PySpark Lab 4 - Movies
8A BSCS MMD  
22nd February 2024

Question 1 (35 marks)
You are provided with the dataset “Movies.csv” that contains information about 1600 movies with
properties such as year, length, main actor and actress, director and popularity. Your task is to read the
file in SPARK RDD and efficiently perform the following queries on the dataset using Spark (Scala).
You must use the spark functionality of persisting data in memory and partitioning. Try to write an
efficient query.


In [1]:
# #Running on Colab
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=6226f6044feeafa78f64b28149c06c23fac95dd89770be162cb1fd57cfb03665
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 35 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package

In [2]:
#  Import the libraries we will need
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

# Create Spark session and ContextRun PySpark.
# create the session
conf = SparkConf().set("spark.ui.port","4050")
# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.appName("DataFrame").config('spark.ui.port', '4050').getOrCreate()
spark

In [4]:
df = spark.read.option("delimiter", ";").option("header", "true").option("encoding", "ISO-8859-1").csv('Movies.csv')
# Show the DataFrame
df.show(truncate=False)

+----+------+--------------------------------------+-------+-----------------------+-------------------+------------------------+----------+------+---------------------+
|Year|Length|Title                                 |Genre  |Actor                  |Actress            |Director                |Popularity|Awards|*Image,,,,,          |
+----+------+--------------------------------------+-------+-----------------------+-------------------+------------------------+----------+------+---------------------+
|INT |INT   |STRING                                |CAT    |CAT                    |CAT                |CAT                     |INT       |BOOL  |STRING,,,,,          |
|1990|111   |Tie Me Up! Tie Me Down!               |Comedy |Banderas, Antonio      |Abril, Victoria    |Almodóvar, Pedro        |68        |No    |NicholasCage.png,,   |
|1991|113   |High Heels                            |Comedy |Bosé, Miguel           |Abril, Victoria    |Almodóvar, Pedro        |68        |No    |Nic

In [ ]:
# 1. Find the total number of movies of each genre. Consider only the movies released after 1960 and have a length greater than 90 minutes.
df = spark.read.option("delimiter", ";").option("header", "true").option("encoding", "ISO-8859-1").csv('Movies.csv')
df.rdd.filter(lambda x:((int(x[0]) if x[0] and x[0].isdigit() else 0 )>1960) and ((int(x[1]) if x[1] and x[1].isdigit() else 0 )>90)).map(lambda x:(x[3],1)).reduceByKey(lambda x,y:x+y).collect()

[('Comedy', 242),
 ('Horror', 26),
 ('Action', 124),
 ('Drama', 434),
 ('Science Fiction', 26),
 ('Mystery', 63),
 ('Music', 21),
 ('War', 15),
 ('Westerns', 5),
 ('Western', 37),
 ('Adventure', 4),
 ('Romance', 1),
 ('Fantasy', 1),
 (None, 1)]

In [ ]:
# 2. For each actress, find the movie in which she acted. You must print the names of the movies and the year of the movie.
df = spark.read.option("delimiter", ";").option("header", "true").option("encoding", "ISO-8859-1").csv('Movies.csv')
df.rdd.map(lambda x: (x[5],(x[0],x[2]))).groupByKey().map(lambda x: (x[0],list(x[1]))).collect()

[('CAT', [('INT', 'STRING')]),
 ('Abril, Victoria',
  [('1990', 'Tie Me Up! Tie Me Down!'), ('1991', 'High Heels')]),
 ('Adams, Brooke',
  [('1983', 'Dead Zone, The'), ('1979', 'Cuba'), ('1978', 'Days of Heaven')]),
 ('Adams, Maud', [('1983', 'Octopussy'), ('1984', 'Target Eagle')]),
 ('Adams, Trudy', [('1989', 'American Angels: Baptism of Blood, The')]),
 ('Adjani, Isabelle', [('1985', 'Subway'), ('1990', 'Camille Claudel')]),
 ('Adolphson, Kristina', [('1982', 'Fanny and Alexander')]),
 ('Aimee, Anouk',
  [('1982', 'Tragedy of a Ridiculous Man'),
   ('1966', 'A Man & a Woman'),
   ('1986', 'A Man & a Woman: Twenty Years Later'),
   ('1966', 'Un Hombre y una Mujer')]),
 ('Aleandro, Norma', [('1985', 'Official Story, The')]),
 ('Alexander, Denise', [('1976', 'Lindbergh Kidnapping Case, The')]),
 ('Algood, Sara', [('1929', 'Blackmail')]),
 ('Allen, Elizabeth', [('1963', "Donovan's Reef")]),
 ('Allen, Joan', [('1988', 'Tucker: The Man & His Dream')]),
 ('Allen, Karen', [('1988', 'Scrooge

In [18]:
# 3. Find the 25 most popular movies released during 1980-1990
df = spark.read.option("delimiter", ";").option("header", "true").option("encoding", "ISO-8859-1").csv('Movies.csv')
df.rdd.filter(lambda x:(int(x[0]) if x[0] and x[0].isdigit() else 0)>=1980 and (int(x[0]) if x[0] and x[0].isdigit() else 0) <=1990).map(lambda x: (int(x[7]) if x[7] else 0,x)).sortByKey(ascending=False).take(25)

[(88,
  Row(Year='1980', Length=None, Title='Happy Birthday to Me', Genre='Horror', Actor='Ford, Glenn', Actress='Anderson, Melissa Sue', Director='Thompson, J. Lee', Popularity='88', Awards='No', *Image,,,,,='glennFord.png,,')),
 (88,
  Row(Year='1989', Length='88', Title='Final Notice', Genre='Mystery', Actor='Gerard, Gil', Actress='Anderson, Melody', Director='Stern, Steven Hilliard', Popularity='88', Awards='No', *Image,,,,,='NicholasCage.png,,')),
 (88,
  Row(Year='1985', Length='55', Title='Gonzo Presents Muppet Weird Stuff', Genre='Comedy', Actor='Cleese, John', Actress='Andrews, Julie', Director=None, Popularity='88', Awards='No', *Image,,,,,='NicholasCage.png,,,')),
 (88,
  Row(Year='1990', Length='105', Title='Guilty by Suspicion', Genre='Drama', Actor='De Niro, Robert', Actress='Bening, Annette', Director='Winkler, Irwin', Popularity='88', Awards='No', *Image,,,,,='NicholasCage.png,,')),
 (88,
  Row(Year='1988', Length=None, Title='Five Corners', Genre='Drama', Actor='Robbin

In [34]:
# 4. Find the average length of the movies of each genre.
df = spark.read.option("delimiter", ";").option("header", "true").option("encoding", "ISO-8859-1").csv('Movies.csv')
def findAverageLength(x):
  genre=x[0]
  records=list(x[1])
  sum=0
  count=0
  for indRecord in records:
    sum=sum+indRecord
    count=count+1

  return genre,(sum/count) if count > 0 else None

df.rdd.map(lambda x: (x[3],int(x[1]) if x[1] and x[1].isdigit() else 0)).groupByKey().map(lambda x: findAverageLength(x)).collect()


[('CAT', 0.0),
 ('Comedy', 92.74545454545455),
 ('Horror', 90.63157894736842),
 ('Action', 98.89268292682927),
 ('Drama', 110.19083969465649),
 ('Mystery', 97.23364485981308),
 ('Science Fiction', 106.47368421052632),
 ('Music', 95.81395348837209),
 ('War', 110.02941176470588),
 ('Westerns', 104.0),
 ('Western', 85.19327731092437),
 (None, 120.5),
 ('Short', 40.0),
 ('Adventure', 119.0),
 ('Crime', 66.0),
 ('Romance', 127.0),
 ('Fantasy', 102.0)]

In [42]:
# 5. Find the lead actor and lead actress pair who have acted in more than one movie together.
df = spark.read.option("delimiter", ";").option("header", "true").option("encoding", "ISO-8859-1").csv('Movies.csv')
df.rdd.map(lambda x: ((x[4],x[5]),1)).reduceByKey(lambda x,y: x+y).filter(lambda x: x[1]>1).collect()


[(('Trintignant, Jean-Louis', 'Aimee, Anouk'), 3),
 (('Björnstrand, Gunnar', 'Andersson, Harriet'), 3),
 (('Hopkins, Anthony', 'Bancroft, Anne'), 2),
 (('Ronet, Maurice', 'Bardot, Brigitte'), 2),
 (('Gere, Richard', 'Basinger, Kim'), 2),
 (('Adolphson, Edvin', 'Bergman, Ingrid'), 2),
 (('Grant, Cary', 'Bergman, Ingrid'), 2),
 (('Eastwood, Clint', 'Bloom, Verna'), 2),
 (('Depardieu, Gérard', 'Deneuve, Catherine'), 2),
 (('Montand, Yves', 'Deneuve, Catherine'), 2),
 (('Sellers, Peter', 'Ekland, Britt'), 2),
 (('Allen, Woody', 'Farrow, Mia'), 2),
 (('Hamill, Mark', 'Fisher, Carrie'), 3),
 (('Gilbert, John', 'Garbo, Greta'), 3),
 (('Stone, Lewis', 'Garbo, Greta'), 2),
 (('Douglas, Melvyn', 'Garbo, Greta'), 2),
 (('Schwarzenegger, Arnold', 'Hamilton, Linda'), 2),
 (('Russell, Kurt', 'Hawn, Goldie'), 2),
 (('Chase, Chevy', 'Hawn, Goldie'), 2),
 (('Ford, Glenn', 'Hayworth, Rita'), 3),
 (('Tracy, Spencer', 'Hepburn, Katharine'), 11),
 (('Allen, Woody', 'Keaton, Diane'), 5),
 (('Hogan, Paul', '

In [46]:
# 5. VERSION 2 (CONSIDERNG BOTH PAIRS) Find the lead actor and lead actress pair who have acted in more than one movie together.
df = spark.read.option("delimiter", ";").option("header", "true").option("encoding", "ISO-8859-1").csv('Movies.csv')
df.rdd.map(lambda x: ((x[4], x[5]), 1)).reduceByKey(lambda x, y: x + y).filter(lambda x: x[1] > 1)\
    .flatMap(lambda x: [(x[0][0], x[0][1]), (x[0][1], x[0][0])]) \
    .distinct() \
    .collect()

[('Trintignant, Jean-Louis', 'Aimee, Anouk'),
 ('Aimee, Anouk', 'Trintignant, Jean-Louis'),
 ('Björnstrand, Gunnar', 'Andersson, Harriet'),
 ('Andersson, Harriet', 'Björnstrand, Gunnar'),
 ('Hopkins, Anthony', 'Bancroft, Anne'),
 ('Bancroft, Anne', 'Hopkins, Anthony'),
 ('Ronet, Maurice', 'Bardot, Brigitte'),
 ('Bardot, Brigitte', 'Ronet, Maurice'),
 ('Gere, Richard', 'Basinger, Kim'),
 ('Basinger, Kim', 'Gere, Richard'),
 ('Adolphson, Edvin', 'Bergman, Ingrid'),
 ('Bergman, Ingrid', 'Adolphson, Edvin'),
 ('Grant, Cary', 'Bergman, Ingrid'),
 ('Bergman, Ingrid', 'Grant, Cary'),
 ('Eastwood, Clint', 'Bloom, Verna'),
 ('Bloom, Verna', 'Eastwood, Clint'),
 ('Depardieu, Gérard', 'Deneuve, Catherine'),
 ('Deneuve, Catherine', 'Depardieu, Gérard'),
 ('Montand, Yves', 'Deneuve, Catherine'),
 ('Deneuve, Catherine', 'Montand, Yves'),
 ('Sellers, Peter', 'Ekland, Britt'),
 ('Ekland, Britt', 'Sellers, Peter'),
 ('Allen, Woody', 'Farrow, Mia'),
 ('Farrow, Mia', 'Allen, Woody'),
 ('Hamill, Mark', 'F

In [68]:
# 6. Find the names of directors who directed movies of both the mystery and drama genres.
df = spark.read.option("delimiter", ";").option("header", "true").option("encoding", "ISO-8859-1").csv('Movies.csv')

mystery=df.rdd.filter(lambda x:(x[3]=='Comedy') and x[6]).map(lambda x:(x[6],x[3])).distinct()
drama=df.rdd.filter(lambda x: (x[3]=='Drama') and x[6]).map(lambda x: (x[6],x[3])).distinct()
mystery.join(drama).collect()

[('Almodóvar, Pedro', ('Comedy', 'Drama')),
 ('Scorsese, Martin', ('Comedy', 'Drama')),
 ('Rosenberg, Stuart', ('Comedy', 'Drama')),
 ('Jewison, Norman', ('Comedy', 'Drama')),
 ('Szabó, István', ('Comedy', 'Drama')),
 ('Daniel, Rod', ('Comedy', 'Drama')),
 ('Gilbert, Lewis', ('Comedy', 'Drama')),
 ('Coppola, Francis Ford', ('Comedy', 'Drama')),
 ('Altman, Robert', ('Comedy', 'Drama')),
 ('Brickman, Paul', ('Comedy', 'Drama')),
 ('Lumet, Sidney', ('Comedy', 'Drama')),
 ('Spielberg, Steven', ('Comedy', 'Drama')),
 ('Bill, Tony', ('Comedy', 'Drama')),
 ('Levinson, Barry', ('Comedy', 'Drama')),
 ('Russell, Ken', ('Comedy', 'Drama')),
 ('Bogdanovich, Peter', ('Comedy', 'Drama')),
 ('Beresford, Bruce', ('Comedy', 'Drama')),
 ('Jarmusch, Jim', ('Comedy', 'Drama')),
 ('Hitchcock, Alfred', ('Comedy', 'Drama')),
 ('Weir, Peter', ('Comedy', 'Drama')),
 ('Eastwood, Clint', ('Comedy', 'Drama')),
 ('Demme, Jonathan', ('Comedy', 'Drama')),
 ('Donaldson, Roger', ('Comedy', 'Drama')),
 ('Annaud, Jean-J

In [60]:
# 6 (B). find names of directors and all the genres of movies they have directed
df = spark.read.option("delimiter", ";").option("header", "true").option("encoding", "ISO-8859-1").csv('Movies.csv')
df.rdd.map(lambda x:(x[6],x[3])).groupByKey().map(lambda x: (x[0],list(x[1]))).collect()

[('CAT', ['CAT']),
 ('Almodóvar, Pedro',
  ['Comedy', 'Comedy', 'Comedy', 'Comedy', 'Comedy', 'Drama']),
 ('Cronenberg, David',
  ['Horror',
   'Drama',
   'Horror',
   'Horror',
   'Horror',
   'Horror',
   'Horror',
   'Horror']),
 ('Lester, Richard', ['Action', 'Action', 'Action']),
 ('Malick, Terrence', ['Drama']),
 ('Glen, John', ['Action', 'Action', 'Adventure']),
 ('Loma, José Antonio de la', ['Action']),
 ('Sebastian, Beverly', ['Drama', 'Action']),
 ('Besson, Luc', ['Drama', 'Drama', 'Drama', 'Drama']),
 ('Nuytten, Bruno', ['Drama']),
 ('Bergman, Ingmar',
  ['Drama',
   'Drama',
   'Drama',
   'Drama',
   'Drama',
   'Drama',
   'Drama',
   'Drama',
   'Drama',
   'Drama',
   'Comedy',
   'Drama',
   'Comedy',
   'Drama',
   'Drama',
   'Comedy',
   'Drama',
   'Drama',
   'Drama',
   'Drama',
   'Drama',
   'Drama',
   'Drama',
   'Drama',
   'Drama',
   'Drama',
   'Drama',
   'Drama',
   'Drama']),
 ('Bertolucci, Bernardo',
  ['Drama', 'Drama', 'Drama', 'Drama', 'Drama', 'D

In [92]:
# 7. Find each director and find the average, max, and min ranking of his movies.
df = spark.read.option("delimiter", ";").option("header", "true").option("encoding", "ISO-8859-1").csv('Movies.csv')
def findInfo(records):
  minX=float('inf')
  maxX=float('-inf')
  sumX=0
  countX=0
  for rec in records:
    if(rec<minX):
      minX=rec
    if(rec>maxX):
      maxX=rec
    sumX = sumX + rec
    countX = countX + 1

  avgX= sumX/countX if countX > 0 else None
  return ('Average Ranking: '+str(avgX),'Maximum Ranking: '+str(maxX),'Minimum Ranking: '+str(minX))
df.rdd.map(lambda x:(x[6],float(x[7]) if x[7] and x[7].isdigit() else 0)).groupByKey().map(lambda x:(x[0],findInfo(list(x[1])))).collect()

[('CAT', ('Average Ranking: 0.0', 'Maximum Ranking: 0', 'Minimum Ranking: 0')),
 ('Almodóvar, Pedro',
  ('Average Ranking: 62.333333333333336',
   'Maximum Ranking: 73.0',
   'Minimum Ranking: 34.0')),
 ('Cronenberg, David',
  ('Average Ranking: 39.375',
   'Maximum Ranking: 79.0',
   'Minimum Ranking: 21.0')),
 ('Lester, Richard',
  ('Average Ranking: 25.0', 'Maximum Ranking: 63.0', 'Minimum Ranking: 6.0')),
 ('Malick, Terrence',
  ('Average Ranking: 14.0', 'Maximum Ranking: 14.0', 'Minimum Ranking: 14.0')),
 ('Glen, John',
  ('Average Ranking: 64.33333333333333',
   'Maximum Ranking: 86.0',
   'Minimum Ranking: 39.0')),
 ('Loma, José Antonio de la',
  ('Average Ranking: 14.0', 'Maximum Ranking: 14.0', 'Minimum Ranking: 14.0')),
 ('Sebastian, Beverly',
  ('Average Ranking: 50.5', 'Maximum Ranking: 73.0', 'Minimum Ranking: 28.0')),
 ('Besson, Luc',
  ('Average Ranking: 22.75', 'Maximum Ranking: 72.0', 'Minimum Ranking: 6.0')),
 ('Nuytten, Bruno',
  ('Average Ranking: 32.0', 'Maximum Ra

Question 2
In this example, you will use the given “Movies.csv” data file. Your task is to write an efficient spark code for finding movies (released after 1970) that might be similar or related to a movie made in the 1960’s (1960-1969).
To keep things simple, for comparing two movies we will use the following fields: Genre, director, lead actor, lead actress.
So, we can say two movies might be similar if they have same genre, director and same lead actor or actress.


In [114]:
df = spark.read.option("delimiter", ";").option("header", "true").option("encoding", "ISO-8859-1").csv('Movies.csv')
# Extracting movies released after 1970. Then using their Genre, director,lead actor , lead actress as key and full record as value to make key-value pairs
movie1970=df.rdd.filter(lambda x: (int(x[0]) if x[0] and x[0].isdigit() else 0)>1970).map(lambda x: ((x[3],x[6],x[4],x[5]),x))
# Extracting movies made in 1960s. Then using their Genre, director,lead actor , lead actress as key and full record as value to make key-value pairs
movie60s=df.rdd.filter(lambda x: (int(x[0]) if x[0] and x[0].isdigit() else 0)>=1960 and (int(x[0]) if x[0] and x[0].isdigit() else 0)<=1969).map(lambda x: ((x[3],x[6],x[4],x[5]),x))
# Join both the rdds based on their key that is Genre, director,lead actor , lead actress & showing movies released after 1970
movie1970.join(movie60s).mapValues(lambda x: x[0]).collect()

[(('Drama', 'Lelouch, Claude', 'Trintignant, Jean-Louis', 'Aimee, Anouk'),
  Row(Year='1986', Length='112', Title='A Man & a Woman: Twenty Years Later', Genre='Drama', Actor='Trintignant, Jean-Louis', Actress='Aimee, Anouk', Director='Lelouch, Claude', Popularity='49', Awards='No', *Image,,,,,='NicholasCage.png,,')),
 (('Drama', 'Lelouch, Claude', 'Trintignant, Jean-Louis', 'Aimee, Anouk'),
  Row(Year='1986', Length='112', Title='A Man & a Woman: Twenty Years Later', Genre='Drama', Actor='Trintignant, Jean-Louis', Actress='Aimee, Anouk', Director='Lelouch, Claude', Popularity='49', Awards='No', *Image,,,,,='NicholasCage.png,,')),
 (('War', None, 'Wayne, John', None),
  Row(Year='1991', Length=None, Title='John Wayne Collection, Red River, The', Genre='War', Actor='Wayne, John', Actress=None, Director=None, Popularity='49', Awards='No', *Image,,,,,='johnWayne.png,,')),
 (('War', None, 'Wayne, John', None),
  Row(Year='1992', Length=None, Title="John Wayne Collector's Limited Edition", G

In [115]:
# Stop the Spark session when you're done
spark.stop()